<a href="https://colab.research.google.com/github/b-rakhshan/Tensorized_Random_Projection/blob/main/Tensorized_Random_Projection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code for Tensorized Random Projection
---
Beheshteh T. Rakhshan, Guillaume Rabusseau

Proceedings of the 23rd International Conference on Artificial Intelligence and Statistics (AISTATS) 2020. 

Requirement:
- Install scilit-tensor-py3 [1] and ttpy [2] for using sktensor (CP decomposition) and Tensor Train decomposition tool-boxes, respectively.

About the code: 
- Implementation of our "CP random projection" and "TT random projection" are found in CP_random_projection and TT_random_projection classes.
- Implementation of classical random projections for comparison with our maps are found in Gaussian_random_projection and Sparse_Gaussian_random_projection classes.

Synthetic data generation:
- Generate random high-dimensional vectors in TT/CP decompositions through random_TT_vector and random_CP_vector functions.

To generate 3 plots in Figure 1 in the paper, run:
- Set ORDER_SIZE to "small", "medium" or "high" to switch between the cases.
- Set INPUT_FORMAT to "TT" or "CP" as an input tensor.
- Set the INPUT_TENSOR_RANK, in our experiment is 10. 









In [ ]:
pip install scikit-tensor-py3

In [ ]:
pip install ttpy

In [ ]:
import sktensor as skt
import tt
import numpy as np
from functools import reduce
from tqdm import tqdm
from sklearn.random_projection import SparseRandomProjection

#Homemade version of matlab tic and toc functions
def tic():
    import time
    global startTime_for_tictoc
    startTime_for_tictoc = time.time()

def toc():
    import time
    if 'startTime_for_tictoc' in globals():
        return time.time() - startTime_for_tictoc

In [ ]:
def innerprodTTCP(tt_cores, cp_cores):
  """
    Compute the inner product between a tensor in TT format and a tensor in CP format

    Parameters
    ---------
    tt_cores: List of ndarrays (3rd order tensor)
      List of cores of TT decomposition of the first tensor (each core is of shape $R \times d_k \times R$)
    cp_cores: List of ndarrays (matrices)
      List of factor matrices of the CP decomposition of the second tensor (each factor matrix is of shape $d_k \times R$)

    Returns
    -------
    Scalar
      Inner product between the two tensors.
  """
  merged_cores = [np.einsum("ijk,jl->ilk",tt_core,cp_core) for tt_core,cp_core in zip(tt_cores,cp_cores)]
  res = np.einsum("ijk,kjl->ijl",merged_cores[0],merged_cores[1])
  for core in merged_cores[2:-1]:
      res = np.einsum("ijk,kjl->ijl",res,core)
  res = np.einsum("ijk,kjl->il",res,merged_cores[-1])
  return res.squeeze()

def innerprodCPCP(cp_cores_1,cp_cores_2):
  """
    Compute the inner product between two tensors in CP formats

    Parameters
    ---------
    cp_cores_1, cp_cores_2: Lists of ndarrays (2nd order tensors -- matrices)
      List of factor matrices of two CP decompositions (each factor matrix is of shape $d_k \times R$)
    
    Returns
    -------
    Scalar
      Inner product between the two tensors.
  """
  merged_cores = [np.einsum("ij,ik->jk",tt_core,cp_core) for tt_core,cp_core in zip(cp_cores_1,cp_cores_2)]
  res = np.einsum("jk,jk->jk",merged_cores[0],merged_cores[1])
  for core in merged_cores[2:-1]:
      res = np.einsum("jk,jk->jk",res,core)
  res = np.einsum("jk,jk",res,merged_cores[-1])
  return res.squeeze()

class TT_random_projection:
  """
  Class implementing the TT tensorized random projection (see def. 1 in the paper [3])
  """
  def __init__(self,dims,ranks,k):  
    """ 
      Initialize an instance of a TT random projection

      Parameters
      ---------
      dims: List of integers
        List of dimension of the input space of the random projection 
      ranks: List of integers or integer
        List of ranks of the TT random projection. If ranks is an integer, the same rank is used for all cores
      k: Integer
        List of dimension of the output space of the random projection (embedding dimension)
    """
    N = len(dims)
    if isinstance(ranks,int):
        ranks = [ranks] * N
    ranks.insert(0,1); ranks[-1]=1
    self.cores = [np.random.normal(0,1/np.power(ranks[j]*ranks[j+1],0.25),[k,ranks[j],dims[j],ranks[j+1]]) 
                  for j in range(N)]
    # self.cores is of shape N x k x R x d x R
    # Generate list of random tt cores and build tt.vectors
    self.tt_vectors = []
    for i in range(k):
        cores = [c[i,:,:] for c in self.cores]
        self.tt_vectors.append(tt.vector.from_list(cores))

    self.N = N
    self.dims = dims
    self.ranks = ranks
    self.k = k
        
  def __call__(self,X):
    """
      Compute the TT random projection of the input tensor X

      Parameters
      ---------
      X : tt.vector, skt.ktensor or ndarray
        X is the input tensor to project using the random projection 

      Returns
      -------
      ndarray
        the random projection, that is a vector (ndarray) of size k
    """
    if isinstance(X,tt.vector): # X is in TT format
        return 1/np.sqrt(self.k)*np.array([tt.dot(v,X) for v in self.tt_vectors])
    elif isinstance(X,skt.ktensor): # X is in CP format
        return 1/np.sqrt(self.k)*np.array([innerprodTTCP(tt.vector.to_list(v),X.U) for v in self.tt_vectors])            
    else: # X is a dense tensor
        X = tt.vector(X)
        return 1/np.sqrt(self.k)*np.array([tt.dot(v,X) for v in self.tt_vectors])
    

class CP_random_projection:
  """
  Class implementing the CP tensorized random projection (see def. 2 in the paper [3])
  """
  def __init__(self,dims,rank,k):
    """ 
      Initialize an instance of a CP random projection

      Parameters
      ---------
      dims: List of integers
        List of dimension of the input space of the random projection 
      ranks: List of integers or integer
        List of ranks of the CP random projection. If ranks is an integer, the same rank is used for all cores
      k: Integer
        List of dimension of the output space of the random projection (embedding dimension)
    """
    N = len(dims)
    self.factors = [np.random.normal(0,1/np.power(rank,1/(2*N)),[k,rank,dims[j]]) 
                  for j in range(N)]
    # self.cores is of shape N x k x R x d x R

    self.cp_vectors = []
    for i in range(k):
        factors = [c[i,:,:].T for c in self.factors]
        self.cp_vectors.append(skt.ktensor(factors))

    self.N = N
    self.dims = dims
    self.rank = rank
    self.k = k
        
  def __call__(self,X):
    """
      Compute the CP random projection of the input tensor X

      Parameters
      ---------
      X : tt.vector, skt.ktensor or kt.dtensor
        X is the input tensor to project using the random projection 

      Returns
      -------
      ndarray
        the random projection, that is a vector (ndarray) of size k
    """
    if isinstance(X,tt.vector): # X is in TT format
        return 1/np.sqrt(k)*np.array([innerprodTTCP(tt.vector.to_list(X),v.U) for v in self.cp_vectors])
    if isinstance(X,skt.ktensor): # X is in CP format
        return 1/np.sqrt(k)*np.array([innerprodCPCP(X.U,v.U) for v in self.cp_vectors])
    elif not isinstance(X,skt.dtensor): # X is a dense tensor
        X = skt.dtensor(X) 
        return 1/np.sqrt(k)*np.array([v.innerprod(X) for v in self.cp_vectors])
    
class Gaussian_random_projection:
  """
    Class implementing the classical Gaussian random projection [4]
  """
  def __init__(self,dims,rank,k):
    """ 
      Initialize an instance of a Gaussian random projection

      Parameters
      ---------
      dims: List of integers
        List of dimension of the input space of the random projection 
        
      ranks: It's 1 in this case
      k: Integer
        List of dimension of the output space of the random projection (embedding dimension)
    """
    N = len(dims)
    self.proj = np.random.normal(0,1,[k,np.prod(dims)])
    self.N = N
    self.dims = dims
    self.k = k
        
        
  def __call__(self,X):
    """
      Compute the Gaussian random projection of the input vector X

      Parameters
      ---------
      X : tt.vector, skt.ktensor
        X is the input vector to project using the random projection 

      Returns
      -------
      ndarray
        the random projection, that is a vector (ndarray) of size k
    """
    if isinstance(X,tt.vector):
        X = tt.vector.full(X)
    if isinstance(X,skt.ktensor):
        X = X.toarray()
    X = X.ravel()
    return 1/np.sqrt(k)*self.proj.dot(X)

    
class Sparse_Gaussian_random_projection:
  """
    Class implementing the sparse Gaussian random projection [5]
  """
  def __init__(self,dims,rank,k):
    """ 
      Initialize an instance of a Sparse Gaussian random projection

      Parameters
      ---------
      dims: List of integers
        List of dimension of the input space of the random projection 
        
      ranks: It's 1 in this case
      k: Integer
        List of dimension of the output space of the random projection (embedding dimension)
    """
    self.proj = SparseRandomProjection(n_components=k)

  def __call__(self,X):
    """
      Compute the Sparse Gaussian random projection of the input vector X

      Parameters
      ---------
      X : tt.vector, skt.ktensor
        X is the input vector to project using the random projection 

      Returns
      -------
      ndarray
        the random projection, that is a vector (ndarray) of size k
    """
    if isinstance(X,tt.vector):
        X = tt.vector.full(X)
    if isinstance(X,skt.ktensor):
        X = X.toarray()
    X = np.expand_dims(X.ravel(),0)
    return self.proj.fit_transform(X)
    
             
def random_TT_vector(dims,ranks):
  """
      Generate random TT vector as an input of CP/TT random projection map in [3]

      Parameters
      ---------
      dims: List of integers
        List of dimension of the input space of the random projection 
      ranks: List of integers or integer
        List of ranks of the TT decomposition. If ranks is an integer, the same rank is used for all cores

      Returns
      -------
      ndarray 
        Random tt vector 
  """
  N = len(dims)
  if not isinstance(ranks,list):
    ranks = [ranks] * N
    ranks.insert(0,1); ranks[-1]=1
  cores = [np.random.normal(0,1,[ranks[j],dims[j],ranks[j+1]]) for j in range(len(dims))]
  return tt.vector.from_list(cores)

def random_CP_vector(dims,rank):
  """
      Generate random CP tensor as an input of CP/TT random projection map in [3]

      Parameters
      ---------
      dims: List of integers
        List of dimension of the input space of the random projection 
      ranks: List of integers or integer
        List of ranks of the CP decomposition. If ranks is an integer, the same rank is used for all cores

      Returns
      -------
      ndarray 
        Random CP tensor 
  """
  return skt.ktensor([np.random.normal(0,1,[d,rank]) for d in dims])


t1 = random_CP_vector([2]*10,5)
t3 = random_CP_vector([2]*10,7)
t2 = random_TT_vector([2]*10,4)

assert(np.allclose(innerprodTTCP(tt.vector.to_list(t2),t1.U), tt.vector.full(t2).ravel().dot(t1.toarray().ravel())))
assert(np.allclose(innerprodCPCP(t3.U,t1.U), t3.toarray().ravel().dot(t1.toarray().ravel())))


### Reproducing Figure 1

The next cell runs the experiment for the small, medium or high-order case (change the ORDER_SIZE variable to switch cases)

The INPUT_FORMAT variable allows to switch between having the input in CP or TT format (rank 10 in both cases).

In [ ]:
# set ORDER_SIZE to "small", "medium" or "high" to reproduce the 3 plots of Figure 1

ORDER_SIZE = "small" #change this variable to "small","medium" or "high" to switch between the cases
INPUT_FORMAT = "TT" # this can be either "TT" or "CP" as an input tensor
INPUT_TENSOR_RANK = 10

if ORDER_SIZE == "small":
  d = 15
  N = 3
  PROJ_LIST = ["CP","Gaussian","TT"]
elif ORDER_SIZE == "medium":
  d = 3
  N = 12
  PROJ_LIST = ["CP","Sparse_Gaussian","TT"]
elif ORDER_SIZE == "high":
  d = 3
  N = 25
  PROJ_LIST = ["CP","Sparse_Gaussian","TT"]


cp_ranks = [4,25,100] # CP random projection ranks
tt_ranks = [2,5,10] # TT random projection ranks
gaussian_ranks = [1]
n_runs = 100 # number of runs

ks = [10,20,40,80,160,320,640,1280,2560] # embedding dimensions



if INPUT_FORMAT == "TT": # generate random rank 10 unit norm TT vector
  X = random_TT_vector([d]*N,INPUT_TENSOR_RANK)
  X = 1/tt.vector.norm(X)*X
  X_norm = tt.vector.norm(X)**2
elif INPUT_FORMAT == "CP": # generate random rank 10 unit norm CP vector
  X = random_CP_vector([d]*N,INPUT_TENSOR_RANK)
  X.U[0] /= X.norm()
  X_norm = X.norm()**2


cp_norms = {}
cp_dists = {}
cp_times = {}
tt_norms = {}
tt_dists = {}
tt_times = {}
gaussian_times = {}
gaussian_norms = {}
gaussian_dists = {}
spgaussian_norms = {}
spgaussian_dists = {}
spgaussian_times = {}


for PROJ in PROJ_LIST:
    if PROJ == "CP":
        times = cp_times
        norms,dists,ranks,random_projection = cp_norms,cp_dists,cp_ranks,CP_random_projection
    elif PROJ == "TT":
        times = tt_times
        norms,dists,ranks,random_projection = tt_norms,tt_dists,tt_ranks,TT_random_projection
    elif PROJ == "Gaussian":
        times = gaussian_times
        norms,dists,ranks,random_projection = gaussian_norms,gaussian_dists,gaussian_ranks,Gaussian_random_projection
    elif PROJ == "Sparse_Gaussian":
        times = spgaussian_times
        norms,dists,ranks,random_projection = spgaussian_norms,spgaussian_dists,gaussian_ranks,Sparse_Gaussian_random_projection
    else:
        continue
        
    for n in tqdm(range(n_runs),desc=PROJ,position=0, leave=True):
        for k in ks:
            for R in ranks:
                tic()
                proj = random_projection([d]*N,R,k)
                v = proj(X)                          
                times[n,k,R] = toc()
                v_norm = np.linalg.norm(v)**2
                norms[n,k,R] = v_norm
                dists[n,k,R] = np.abs(v_norm/X_norm - 1)




The next cell plots the result

In [ ]:
import numpy as np
import matplotlib.pylab as plt

from matplotlib.pyplot import cm

def dic2list(dic, R, ks, n_runs, agg_func):
  d = {}
  for k in ks:
    d[R,k] = agg_func([dic[n,k,R] for n in range(n_runs)])
  return [d[R,k] for k in ks]

def plot_results(data,ranks,ks,n_runs,func,legend,proj_type,plot_errors=False):
    colors="brgm"
    markers="od*<>^v"
    if proj_type=="CP":
        line = "--" 
    elif proj_type=="TT":
        line = "-" 
    else:
        line = ':'; colors=["k"]
    for R,color,marker in zip(ranks[::-1],colors,markers):
        vals = dic2list(data,R,ks,n_runs,func)
        if plot_errors:
            stds = dic2list(data,R,ks,n_runs,np.std)
            plt.errorbar(ks,vals,yerr=0.2*np.array(stds),capsize=5,c=color,ls=line)
        else:
            plt.plot(ks,vals,c=color,ls=line)
        legend.append(proj_type + ("" if "Gaussian" in proj_type else f"({R})"))
            
norms = {"CP":cp_norms, "TT":tt_norms, "Gaussian":gaussian_norms, "Sparse_Gaussian":spgaussian_norms}
times = {"CP":cp_times, "TT":tt_times, "Gaussian":gaussian_times, "Sparse_Gaussian":spgaussian_times}
dists = {"CP":cp_dists, "TT":tt_dists, "Gaussian":gaussian_dists, "Sparse_Gaussian":spgaussian_dists}
ranks = {"CP":cp_ranks, "TT":tt_ranks, "Gaussian":[1], "Sparse_Gaussian":[1]}


plt.figure()
legend = []
for proj_type in PROJ_LIST:
    plot_results(dists[proj_type],ranks[proj_type],ks,n_runs,np.mean,legend,proj_type)
plt.legend(legend,loc=1,ncol=2)
plt.yscale("log")
plt.title(ORDER_SIZE + " case: distortions")
plt.xlabel("Embedding dimension")
plt.ylabel("Distortion ratio")

# plot running time
plt.figure()
legend = []
for proj_type in PROJ_LIST:
    plot_results(times[proj_type],ranks[proj_type],ks,n_runs,np.mean,legend,proj_type)
plt.legend(legend,loc=1,ncol=2)
plt.yscale("log")
plt.title(ORDER_SIZE + " case: running times")
plt.xlabel("Embedding dimension")
plt.ylabel("Running time")

# Refrences

[1] Maximilian Nickel, Evert Rol. Python scikit-tensor toolbox: a Python module for multilinear algebra and tensor factorizations. Available online, URL https://github.com/evertrol/scikit-tensor-py3.

[2] Ivan Oseledets, Vladimir Kazeev, et al. Python tt-toolbox2.2:   Fast  multidimensional  array  operations  in  tt-format.  Available online, June 2014. URL https://github.com/oseledets/ttpy.

[3] Rakhshan, Beheshteh, and Guillaume Rabusseau. "Tensorized random projections." International Conference on Artificial Intelligence and Statistics. PMLR, 2020.

[4] Johnson, William B., and Joram Lindenstrauss. "Extensions of Lipschitz mappings into a Hilbert space." Contemporary mathematics 26.189-206 (1984).

[5] Achlioptas, Dimitris. "Database-friendly random projections." Proceedings of the twentieth ACM SIGMOD-SIGACT-SIGART symposium on Principles of database systems. 2001.